In [ ]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats.mstats import gmean 
from datetime import datetime as dt

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


iFirstFactYear=2005
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

In [ ]:
class finance:
    pdfWork=None
    
    lstYearCodes=['CPIAv','KeyRate','developers_ss_s','guarantee_coverage','loans_secured',
                  'rw_loans_secured','rw_loans_not_secured','h1','ROE','dom_market_share',
                  'contributions_rate_ASV','contributions_rate_FOR','Vvody','VvodyMKD_private',
                  'ProjectsPrivate_x','ProjectsPrivate_escrow_s','ConstrSales_s','seb1m_full_x',
                  'price1mperv_x','loans_and_ref_vol','loans_vol_MKD_perv'] # список экзогенных и фактических данных
 
    def __init__(self):
        assert False, 'you can\'t create variables of repay_e class!'
    
    def MakeWorkFrame():
        ''' Загрузка исходных данных
            Исходные данные:
                Фактические значения - из базы данных year.sqlite3;
                Экзогенные - из базы данных exog_year.sqlite3
                Задаваемые вручную - из базы данных exog_param.sqlite3
                Результаты других моделей - из базы данных svod.sqlite3. 
        '''
        
        pdfAct=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWork)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['KeyRate'], multiplier=1e2)
        .pipe(common.scale, list_fields=['loans_vol_MKD_perv'], multiplier=1e-3)) 
        
        pdfExog=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkEx)
        .pipe(common.make_frame))
        
        PdfExogHandle=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkExH)
        .pipe(common.make_frame))
        
        pdfSvod=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkSvod)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['loans_and_ref_vol','loans_vol_MKD_perv'], multiplier=1e3))
        
        finance.pdfWork=pdfAct.combine_first(pdfExog).combine_first(PdfExogHandle).combine_first(pdfSvod)

        finance.pdfWork['ProjectsPrivate_escrow_x']=finance.pdfWork['ProjectsPrivate_x']*finance.pdfWork['ProjectsPrivate_escrow_s']/100
        
        return finance.pdfWork

In [ ]:
finance.pdfWork = finance.MakeWorkFrame()
finance.pdfWork

In [ ]:
class _calc_finance():
    """класс функций рассчитывающих показатели блока "Финансирование ЖС" """
    
    def _calc_ddy_vvody(series):
        """расчеты запуска  ДДУ"""
        return series['ProjectsPrivate_x']-series['ProjectsPrivate_escrow_x']
    
    def _calc_ddy_disps(series):
        return series['ConstrSales_s']*series['price1mperv_x']*series['ddy_vvody']
    
    def _calc_ddy_sales(series):
        params=[0.25,0.5,0.25]
        print(series)
        try:
            return np.sum(series*params)
        except:
            return None
        
    def _calc_perv_in_ipot(series):
        return series['loans_and_ref_vol','loans_vol_MKD_perv'].sum()

#     def _calc_escrow_dsclosure(series):
#         return series['ConstrSales_s']*series['price1mperv_x']*series['ddy_vvody']



In [ ]:
finance.pdfWork['ddy_vvody'] = finance.pdfWork.apply(_calc_finance._calc_ddy_vvody, axis=1)
finance.pdfWork['ddy_disps'] = finance.pdfWork[['ConstrSales_s', 'price1mperv_x', 'ddy_vvody']].shift(2).apply(_calc_finance._calc_ddy_disps, axis=1)
finance.pdfWork.loc[2021,'ddy_disps'] = finance.pdfWork.loc[2021,'ddy_sales']+finance.pdfWork.loc[2020,'ddy_portf']

# finance.pdfWork['ddy_sales'] = finance.pdfWork['price1mperv_x'].rolling(window=3).apply(_calc_finance._calc_ddy_sales)
finance.pdfWork['ddy_sales'] = finance.pdfWork['ConstrSales_s']*(finance.pdfWork['price1mperv_x'] * finance.pdfWork['ddy_vvody']).rolling(window=3, win_type='triang').sum(fftbins=False)/2


finance.pdfWork.loc[2019,'ddy_portf']=3157.7

    
finance.pdfWork.loc[2020:,'ddy_portf'] = finance.pdfWork.loc[2020:, 'ddy_sales']-finance.pdfWork.loc[2020:,'ddy_disps']+finance.pdfWork.shift(1).loc[2020:,'ddy_portf']

finance.pdfWork[['ddy_vvody', 'ddy_disps', 'price1mperv_x', 'ddy_sales', 'ddy_portf']]
# finance.pdfWork['perv_in_ipot_s'] = finance.pdfWork.apply(_calc_finance._calc_perv_in_ipot, axis=1)

